In [1]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    # Right now the max users is hard coded as 10
    max_users = 10
    
    return "https://api.twitter.com/2/users/{}/followers?max_results={}".format(user_id,max_users)

def get_params():
    return {'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location'}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params):
    # params['next_token'] = next_token # param object recieved form the create_url function
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response


ridge_followers= main()

200


In [2]:
ridge_followers

{'data': [{'username': 'DrMichelleWood',
   'name': 'Michelle Wood, DO',
   'location': 'Winston Salem NC',
   'created_at': '2022-05-24T12:22:15.000Z',
   'description': 'Wife. ❤️ Physician. 👩🏼\u200d⚕️ WVSOM Alumna👩\u200d🎓 Women’s Health and LGBT 🏳️\u200d🌈 Advocate.',
   'id': '1529075188297609217',
   'public_metrics': {'followers_count': 7,
    'following_count': 70,
    'tweet_count': 5,
    'listed_count': 0},
   'verified': False},
  {'username': 'AugWid',
   'name': 'AugWid',
   'location': 'Boulder, CO',
   'created_at': '2021-12-18T17:33:29.000Z',
   'description': 'Porque no?',
   'id': '1472258647371784205',
   'public_metrics': {'followers_count': 19,
    'following_count': 207,
    'tweet_count': 100,
    'listed_count': 4},
   'verified': False},
  {'username': 'phase_cycling',
   'name': 'PHASE Cycling',
   'created_at': '2022-04-29T16:57:03.000Z',
   'description': 'An amateur bicycle racing and touring team located in the Wash. DC Metro Area. Est 2013\n#phasecycling #p

In [3]:
df=pd.DataFrame.from_dict(ridge_followers['data'])
df

,username,name,location,created_at,description,id,public_metrics,verified
0,DrMichelleWood,"Michelle Wood, DO",Winston Salem NC,2022-05-24T12:22:15.000Z,Wife. ❤️ Physician. 👩🏼‍⚕️ WVSOM Alumna👩‍🎓 Wome...,1529075188297609217,"{'followers_count': 7, 'following_count': 70, ...",False
1,AugWid,AugWid,"Boulder, CO",2021-12-18T17:33:29.000Z,Porque no?,1472258647371784205,"{'followers_count': 19, 'following_count': 207...",False
2,phase_cycling,PHASE Cycling,NaN,2022-04-29T16:57:03.000Z,An amateur bicycle racing and touring team loc...,1520084651016179712,"{'followers_count': 8, 'following_count': 254,...",False
3,DaemonicVoid,Daemonic Void,NaN,2013-03-16T00:15:25.000Z,Grey meow meow with a big floofy tail. He/Him....,1271013060,"{'followers_count': 14, 'following_count': 229...",False
4,cxchris,Chris,Wooten - ATX,2009-06-05T05:43:55.000Z,"""Flip on your TV,\nAnd try to make sense out o...",44831529,"{'followers_count': 83, 'following_count': 287...",False
5,Bflo_Bud,Bud Abraham 🇺🇦,NaN,2012-04-02T00:37:49.000Z,Always look on the bright side of life!,543058410,"{'followers_count': 127, 'following_count': 34...",False
6,jwynn9154,James Wynn,"Ridgeland, MS",2018-01-18T14:10:07.000Z,"Transplant surgeon, dialysis access surgeon. w...",953992904321757185,"{'followers_count': 372, 'following_count': 11...",False
7,LeannWarren,Leann Warren,NaN,2009-02-14T21:38:07.000Z,Cyclist. Former Runner. Pug Lover. Oregon Duck.,20873130,"{'followers_count': 313, 'following_count': 18...",False
8,jFursty,Joe Furstenberg,NaN,2009-04-20T21:35:40.000Z,"Tweets are my own, RT do not equal endorsement...",33673431,"{'followers_count': 504, 'following_count': 23...",False
9,andlivs2,Andy,NaN,2022-01-31T16:34:44.000Z,🌱FBLC,1488188967296151554,"{'followers_count': 203, 'following_count': 11...",False


In [4]:
df.description[1]

'Porque no?'

## Find likes

In [5]:

def create_liked_url():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    tweet_fields = "tweet.fields=text,author_id"
    user_fields = "user.fields=username"
    # Be sure to replace your-user-id with your own user ID or one of an authenticating user
    # You can find a user ID by using the user lookup endpoint
    id = 3413455348
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    liked_url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    return liked_url, tweet_fields, user_fields

def connect_to_liked_endpoint(liked_url, tweet_fields):
    response = requests.request(
        "GET", liked_url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    liked_url, tweet_fields, user_fields = create_liked_url()
    json_response = connect_to_liked_endpoint(liked_url, tweet_fields)
    return json_response

liked_texts =main()
liked_texts

200


{'data': [{'author_id': '16949790',
   'id': '1506734904549249027',
   'text': '@sdbeachgirl9 @philgaimon @ridgesupply'},
  {'author_id': '250376708',
   'id': '1506728125706559491',
   'text': '@philgaimon Love the socks 🌈 as a proud supporter, a fan of rainbows, and a bonafide cycling sock nut 🤪 Where can I get some? 🤩🤔'},
  {'author_id': '69469164',
   'id': '1506688981005746184',
   'text': '@MBTallentire @ridgesupply'},
  {'author_id': '876379387112849417',
   'id': '1506688850239934469',
   'text': '@philgaimon where do i get a pair of these? 🌈'},
  {'author_id': '2838151979',
   'id': '1506738461939838982',
   'text': '@philgaimon If this was a @ridgesupply ad then it totally worked. XL was still in stock!'},
  {'author_id': '16984098',
   'id': '1506735997220003848',
   'text': '@philgaimon Love those @ridgesupply socks!'},
  {'author_id': '43325832',
   'id': '1506718283101360142',
   'text': "@philgaimon Are those socks from @ridgesupply ? If so, I have 7 pairs in different c

## Retweets

In [6]:

def create_url():
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=created_at,description"
    # You can replace the ID given with the Tweet ID you wish to lookup Retweeting users for
    # You can find an ID by using the Tweet lookup endpoint
    id = "1437987598790582277"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets/{}/retweeted_by".format(id)
    return url, user_fields


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RetweetedByPython"
    return r

def connect_to_endpoint(url, user_fields):
    response = requests.request("GET", url, auth=bearer_oauth, params=user_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url, user_fields = create_url()
    json_response = connect_to_endpoint(url, user_fields)
    return json_response

retweets = main()
retweets



200


{'data': [{'description': '',
   'name': 'Gregg Schwinn',
   'created_at': '2009-08-12T04:39:34.000Z',
   'id': '64939212',
   'username': '605erLBM'},
  {'description': 'PhD, UC Berkeley: School Psychologist \n#Warriors 🏀Fan, Music Lover, #Phish Phan\n#VOTEBLUE2022 #BLM #RESIST \nWill chat w/MAGAs but Lies/Insults=Blocked, No DMs!',
   'name': 'Lady Cybele',
   'created_at': '2016-10-15T04:03:33.000Z',
   'id': '787141837286875137',
   'username': 'lady_cybele'},
  {'description': '',
   'name': 'Mctrish',
   'created_at': '2019-04-09T16:10:36.000Z',
   'id': '1115648212231016450',
   'username': 'Mctrish5'},
  {'description': 'science instructor, watercolor hobby, clarinet student.',
   'name': 'larnelson',
   'created_at': '2008-06-11T21:00:35.000Z',
   'id': '15089176',
   'username': 'larry_nelson_'},
  {'description': '',
   'name': 'BonnieH',
   'created_at': '2014-05-24T04:53:46.000Z',
   'id': '2519518177',
   'username': 'Diamonddog17'},
  {'description': 'Your opinion of you

In [7]:
df_retweet=pd.DataFrame. from_dict(retweets['data'])
df_retweet


,description,name,created_at,id,username
0,,Gregg Schwinn,2009-08-12T04:39:34.000Z,64939212,605erLBM
1,"PhD, UC Berkeley: School Psychologist \n#Warri...",Lady Cybele,2016-10-15T04:03:33.000Z,787141837286875137,lady_cybele
2,,Mctrish,2019-04-09T16:10:36.000Z,1115648212231016450,Mctrish5
3,"science instructor, watercolor hobby, clarinet...",larnelson,2008-06-11T21:00:35.000Z,15089176,larry_nelson_
4,,BonnieH,2014-05-24T04:53:46.000Z,2519518177,Diamonddog17
...,...,...,...,...,...
89,,T. F. H.,2020-07-03T19:06:26.000Z,1279129295550664706,TFH71222518
90,"☆ Inspired by Art, Colors & Sleep ☆ #VoteBlue2...",🎨📿ArtsyStuffMaker 🇺🇦🌻,2015-10-04T07:23:18.000Z,3778668494,ArtsyStuffMaker
91,Gentleman. Please follow my twitter. I'll foll...,Bobby Riner,2020-01-01T19:29:41.000Z,1212455669002956800,bobby_riner
92,"Sports, politics, religion, history, movies, T...",Lynn G. Henning,2011-09-16T18:45:37.000Z,374676084,Lynn_Henning
